<a href="https://colab.research.google.com/github/techpvk/EVR1/blob/main/s6/EVR1_S6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

![image.png]()

In [25]:
#Approch1
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=0)
        self.conv2_bn = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 32, 3)
        self.conv4 = nn.Conv2d(32, 32, 3)
        self.conv5 = nn.Conv2d(32, 64, 3)
        self.conv3_bn = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv6 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(64, 10)
    def forward(self, x):
        x = self.pool1(self.conv2_bn(F.relu(self.conv2(F.relu(self.conv1(x))))))
        x = self.pool2(self.conv3_bn(F.relu(self.conv5(F.relu(self.conv4(F.relu(self.conv3(x))))))))
        x = F.relu(self.conv6(x))
        x = x.view(-1, 64)
        x = F.relu(self.fc1(x))
        return F.log_softmax(x)

In [26]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
            Conv2d-2           [-1, 16, 24, 24]           2,320
       BatchNorm2d-3           [-1, 16, 24, 24]              32
         MaxPool2d-4           [-1, 16, 12, 12]               0
            Conv2d-5           [-1, 32, 10, 10]           4,640
            Conv2d-6             [-1, 32, 8, 8]           9,248
            Conv2d-7             [-1, 64, 6, 6]          18,496
       BatchNorm2d-8             [-1, 64, 6, 6]             128
         MaxPool2d-9             [-1, 64, 3, 3]               0
           Conv2d-10             [-1, 64, 1, 1]          36,928
           Linear-11                   [-1, 10]             650
Total params: 72,602
Trainable params: 72,602
Non-trainable params: 

<ipython-input-25-2bec6ea665f7>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [27]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [28]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [29]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-25-2bec6ea665f7>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.043232690542936325 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.70it/s]



Test set: Average loss: 0.0762, Accuracy: 9756/10000 (98%)



loss=0.0600108802318573 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.39it/s]



Test set: Average loss: 0.0349, Accuracy: 9885/10000 (99%)



loss=0.01899762451648712 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 26.04it/s]


...


Test set: Average loss: 0.0179, Accuracy: 9944/10000 (99%)



loss=0.002903282642364502 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.13it/s]



Test set: Average loss: 0.0226, Accuracy: 9930/10000 (99%)



loss=0.0007465691305696964 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.63it/s]



Test set: Average loss: 0.0176, Accuracy: 9952/10000 (100%)

